# Dysfunctional seed dispersal in the endemic flora of Madagascar

* Language: R
* Methods: linear regression, multivariate imputation by chained equations (MICE), spatial analyses
* Associated publication: (in prep)

In [3]:
library(raster);
library(igraph);

<br>
<br>
## Overlap between plant and lemur distributions

We want to answer the following questions:								
1) How many lemurs are able to disperse a given species s?	
2) How many lemurs co-occur with species s?		
3) How many lemurs are able to disperse species s AND co-occur with it?

##### Read distribution data of plant species

In [ ]:
dat <- read.table("Plant_coord.txt", sep="\t", header=TRUE)	

##### Read all vegetation rasters (lower resolution)

In [ ]:
matveg <- expand.grid(For=c("no","yes"), Bush=c("no","yes"), Grass=c("no","yes"), Anthr=c("no","yes")) # combination matrix
matveg <- matveg[-1,]        # remove first line (with "no" only)


for (c in 1:nrow(matveg)) {
	combveg <- names(matveg[, matveg[c,]=="yes", drop=FALSE]) 
	combname <- paste(combveg, collapse="")
	namerasterveg <- paste("raster", combname, sep="")
	namerastervegAggprtif <- paste(namerasterveg, "Aggpr.tif", sep="") 
	assign(namerasterveg, raster(namerastervegAggprtif))
	rasterveg <- get(namerasterveg)
	rasterveg[rasterveg==0] <- NA
	assign(namerasterveg, rasterveg)
}


# Prepare base map (values=0) to create rasters for plants
rasterBase <- raster("rasterBaseAggpr.tif")
proj4string(rasterBase) <- CRS("+proj=longlat +ellps=WGS84 +datum=WGS84")

# Area of each cell of the raster (km²)
areacell <- area(rasterBase)

##### Read all lemur rasters

In [ ]:
lemur <- read.csv("Dispersers.csv")
lemur <- subset(lemur, Taxon=="Primate")
lemur$Disperser = factor(lemur$Disperser)

lemursp <- lemur$Disperser
for (l in seq_along(lemursp)) {
#l=2
	namerasterlem <- paste("raster", lemursp[l], sep="")
	namerasterlemtif <- paste("raster",lemursp[l], ".tif", sep="") 
	assign(namerasterlem, raster(namerasterlemtif))
}

##### Prepare containers for data

In [ ]:
distrAreaTot <- list()
distrCoLemurTot <- list()
distrDispLemurTot <- list()
distrDispCoLemurTot <- list()

##### Loop through every imputed dataset

In [ ]:
for (d in 1:20) {
	######## Endozoochorous plant species
	namefiled <- paste("traitveg", d, ".csv", sep="")
	traitendo <- read.csv(namefiled)
	traitendo <- subset(traitendo, SyndAB2 == "Biotic")
	speciesendo <- traitendo$.id
	rownames(traitendo) <- speciesendo


	#========================== For each plant species s
    ## Prepare containers for data
	distrArea <- list()
	distrCoLemur <- list()
	distrDispLemur <- list()
	distrDispCoLemur <- list()


	for (s in seq_along(speciesendo)) {

		#========================== CREATE A RASTER FOR EACH PLANT SPECIES S (=rasterpls) ====================

		speciesendos <- as.character(speciesendo[s])
		dats <- subset(dat, species==speciesendos)


		if (nrow(dats)==0) {
		distrArea[[s]] <- "no gbif data"
		distrCoLemur[[s]] <- "no gbif data"
		distrDispLemur[[s]] <- "no gbif data"
		distrDispCoLemur[[s]] <- "no gbif data"
		} else {  


			######## Create rasters for plants (Transform points into a Raster object)
			x <- dats$decimalLongitude
			y <- dats$decimalLatitude
			coord <- data.frame(x,y)
			rasterpls <- rasterize(coord, rasterBase, field=1)


			######## Keep, in rasters, only vegetation types associated with species s (to remove any artefact)
			traitendos <- subset(traitendo, .id==speciesendo[s])                 # species s in dataset d
			vegtypes <- names(traitendos[, traitendos[1,]=="yes", drop=FALSE])   # vegetation types associated with species s
		
			if (length(vegtypes) != 0) {                              # some species have an estimated "no" for every vegetation type (except Veg_Mang not imputed)
			combnames <- paste(vegtypes, collapse="") 
			namerasterveg <- paste("raster", combname, sep="")        # name of the corresponding vegetation raster
			rasterveg <- get(namerasterveg)
			projection(rasterpls) <- projection(rasterveg)
			rasterpls <- mask(rasterpls, rasterveg)                   # intersection between plant species raster and vegetation raster
			} 

			######## Calculate and save total area
			values(rasterpls)[is.na(values(rasterpls))] <- 0          # replace NAs by 0, otherwise the multiplication will not be computed
			values(rasterpls) <- values(areacell) * values(rasterpls) # multiply the cell areas with Raster object with presence/absence values
			distrArea[[s]] <- cellStats(rasterpls, 'sum')             # potential distribution area of the plant species s (in km²)
			rasterpls[rasterpls==0] <- NA



			#========================== OVERLAP BETWEEN PLANT S AND LEMUR RASTERS =====================================

			######## Identify lemurs co-occurring with plant species s
			cooclem <- data.frame(CoLemur=rep(NA, 20), CoArea=rep(NA, 20))

			for (l in seq_along(lemursp)) {
				namerasterlem <- paste("raster", lemursp[l], sep="")
				rasterlemur <- get(namerasterlem)
				cooccur <- mask(rasterlemur, rasterpls)               # Co-occurrence
				values(cooccur)[is.na(values(cooccur))] <- 0          # replace NAs by 0, otherwise the multiplication will not be computed
				values(cooccur) <- values(areacell) * values(cooccur) # multiply the cell areas with Raster object with presence/absence values
				cooclem[l, "CoArea"] <- cellStats(cooccur , 'sum')    # Area of co-occurrence (km²) (may be 0)
				cooclem[l, "CoLemur"] <- as.character(lemursp[l])     # Save names of lemurs in a dataframe
			}
			distrCoLemur[[s]] <- cooclem
			distrCoLemur[[s]] <- subset(distrCoLemur[[s]], CoArea>0)  # Save names of co-occurring lemurs + CoArea (km²)
			if (nrow(distrCoLemur[[s]])==0) {
			distrCoLemur[[s]] <- "no disperser"}


			######## Identify lemurs able to disperse species s according to seed width
			SeWi <- traitendo[as.character(speciesendo[s]),"SeWi"]   # Seed width of species s
			lemur$disp <- ifelse (SeWi <= lemur$MaxSeWi, 1, 0)       # Able to disperse? 1/0
			lemurdisp <- (subset(lemur, disp=="1"))$Disperser        # Species of lemurs able to disperse species s
			nblemurdisp <- length(lemurdisp)                         # Nb of lemurs able to disperse species s
			distrDispLemur[[s]] <- lemurdisp                         # Save names of lemurs able to disperse species s


			######## Identify lemurs able to disperse species s according to seed width and co-occurring with it
			cooclemdisp <-data.frame(CoLemurDisp=rep(NA, length(cooclem)), CoAreaDisp=rep(NA, length(cooclem)))
			for (ld in seq_along(lemurdisp)) {
				namerasterlemurdisp <- paste("raster", lemurdisp[ld], sep="")
				rasterlemur <- get(namerasterlemurdisp)
				cooccur <- mask(rasterlemur,rasterpls)                      # Co-occurrence
				values(cooccur)[is.na(values(cooccur))] <- 0                # replace NAs by 0, otherwise the multiplication will not be computed
				values(cooccur) <- values(areacell) * values(cooccur)       # multiply the cell areas with Raster object with presence/absence values
				cooclemdisp[ld, "CoAreaDisp"] <- cellStats(cooccur,'sum')   # Save area
				cooclemdisp[ld, "CoLemurDisp"] <- as.character(lemurdisp[ld]) # Save names of lemurs
			}
			distrDispCoLemur[[s]] <- cooclemdisp
			distrDispCoLemur[[s]] <- subset(distrDispCoLemur[[s]], CoAreaDisp>0)
		} # end ifelse
        
names(distrArea) <- as.character(speciesendo[1:s])
names(distrCoLemur) <- as.character(speciesendo[1:s])
names(distrDispLemur) <- as.character(speciesendo[1:s])
names(distrDispCoLemur) <- as.character(speciesendo[1:s])



	} # end loop species s


## Save data in a list that will contain the results for every imputed dataset
distrAreaTot[[d]] <- distrArea
distrCoLemurTot[[d]] <- distrCoLemur
distrDispLemurTot[[d]] <- distrDispLemur
distrDispCoLemurTot[[d]] <- distrDispCoLemur


## Save data
namedistrAreaTot <- paste("distrAreaTot", d, ".rda", sep="")
namedistrCoLemurTot <- paste("distrCoLemurTot", d, ".rda", sep="")
namedistrDispLemurTot <- paste("distrDispLemurTot", d, ".rda", sep="")
namedistrDispCoLemurTot <- paste("distrDispCoLemurTot", d, ".rda", sep="")

save(distrAreaTot, file=namedistrAreaTot)
save(distrCoLemurTot, file=namedistrCoLemurTot)
save(distrDispLemurTot, file=namedistrDispLemurTot)
save(distrDispCoLemurTot, file=namedistrDispCoLemurTot)


} # end loop imputed datasets d

## Give proper name to each list element
names(distrAreaTot) <- 1:d
names(distrCoLemurTot) <- 1:d
names(distrDispLemurTot) <- 1:d
names(distrDispCoLemurTot) <- 1:d

Note: We could perform the spatial analyses on 3018 endozoochorous species for which GBIF had georeferenced records.